In [3]:
from mastodon import Mastodon
import pandas as pd
mastodon = Mastodon(
    access_token = '', # from settings/applications make application + get token
    api_base_url = '' # e.g. 'https://chaos.social'
)

In [88]:
home_timeline = mastodon.timeline_home(limit = 40)

40


In [18]:
account_activity = mastodon.fetch_remaining(mastodon.account_statuses(mastodon.me().id, limit=40))

In [34]:
account_favourites = mastodon.fetch_remaining(mastodon.favourites(limit = 40))

In [93]:
account_activity_count = pd.json_normalize(account_activity)["reblog.account.username"].value_counts()

In [130]:
featured_tag_count = pd.json_normalize(mastodon.featured_tag_suggestions()).set_index("name").apply(lambda x: x["history"][0]["uses"], axis=1).sort_values(ascending=False)

In [64]:
favourite_counts = pd.json_normalize(account_favourites)["account.username"].value_counts()

In [90]:
home_timeline_df = pd.json_normalize(home_timeline)

In [95]:
def extract_count_feature(row, count_series: pd.Series):
    if not "account.username" in row:
        return 0
    if row["account.username"] in count_series:
        return count_series[row["account.username"]]
    else:
        return 0

def extract_like_feature(row):
    return extract_count_feature(row, favourite_counts)

def extract_activity_feature(row):
    return extract_count_feature(row, account_activity_count)




In [96]:
home_timeline_df["like_feature"] = home_timeline_df.apply(extract_like_feature, axis=1)
home_timeline_df["activity_feature"] = home_timeline_df.apply(extract_activity_feature, axis=1)

In [ ]:
from IPython.display import display, HTML
home_timeline_df["score"] = home_timeline_df["like_feature"] + home_timeline_df["activity_feature"]
home_timeline_df.sort_values("score", ascending=False).head(40).apply(lambda x: 
    display(HTML(
        f"<b>{x['account.username']}<b>" 
        + x.content 
        + str(" ".join(pd.Series(x['media_attachments']).apply(lambda x: f"<br><img src='{x['preview_url']}'/>" if "preview_url" in x else "").to_list()))
        + (x['url'] if x['url'] is not None else "")
    )), axis=1)